In [16]:
%pylab inline
from sys import platform
import pandas as pd
from datetime import datetime
import text2emotion

if platform == "win32":
    path = 'C:/Users/olive/'
elif platform == "darwin":
    path = '~/'
    
pd.options.mode.chained_assignment = None  # default='warn'
    
%run SentimentAnalysis.ipynb
%run GeoFeatures.ipynb

Populating the interactive namespace from numpy and matplotlib


/Users/ollyjc/Documents/GitHub/COVID-19-Sentiment-and-Compliance-Analyzer/.venv/lib/python3.9/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [ ]:
# EXPLORATORY ANALYSIS

from wordcloud import WordCloud

def cloud_gen(text):
    long_string = ','.join(list(text.values))

    wordcloud = WordCloud(background_color="white",
                          max_words=5000,
                          contour_width=3,
                          contour_color='steelblue'
                         )

    return wordcloud.generate(long_string)

In [ ]:
### READS IN RAW REDDIT DATA FROM UNIVERSITY ONEDRIVE
### INTO COMMENT AND SUBMISSION DATAFRAMES

c_df = pd.read_csv(path + 'OneDrive - Cardiff University/Individual Project/raw data/comments.csv', index_col ='id')
s_df = pd.read_csv(path + 'OneDrive - Cardiff University/Individual Project/raw data/submissions.csv', index_col ='id')

In [2]:
### LOADS PROCESSED DATA

c_df = pd.read_pickle(path + 'OneDrive - Cardiff University/Individual Project/processed data/comments.csv')
s_df = pd.read_pickle(path + 'OneDrive - Cardiff University/Individual Project/processed data/submissions.csv')

In [ ]:
### ATTEMPT TO GEOCODE LOCATIONS FOR EACH SUBMISSION WHERE POSSIBLE

subs2geocode = s_df.sample(500)
subs2geocode['coords'] = get_captions([*subs2geocode['title'].values])

In [3]:
pt_wordcloud = cloud_gen(c_df['processed_text'])
pt_wordcloud.to_image()

NameError: name 'cloud_gen' is not defined

In [ ]:
### PROCESSING OF RAW COMMENT AND SUBMISSION DATA

# Create a datetime from the 'created_utc' column of each row
c_df['datetime'] = [datetime.fromtimestamp(i) for i in c_df['created_utc']]
s_df['datetime'] = [datetime.fromtimestamp(i) for i in s_df['created_utc']]

In [ ]:
# Perform sentiment analysis on each comment and return the sentiment and compound score
c_df['sentiment'], c_df['compound_score'] = sentiment_analysis([*c_df['body']])

In [41]:
### QUERY DATAFRAME FOR NON NEUTRAL COMMENTS

c_df = c_df.query('sentiment != "neutral"')

In [27]:
# TEXT PREPROCESSING
import re

# REMOVING PUNCTUATION
c_df['processed_text'] = c_df['body'].map(lambda x: re.sub('[,\.!?]', '', x))

# SET ALL TEXT TO LOWERCASE
c_df['processed_text'] = c_df['processed_text'].map(lambda x: x.lower())

c_df['processed_text'].head()

id
fhaw2m3    Worrying part of the UK known cases are all pe...
fhawk4d    Surely it’s going to hit at all large multi-na...
fhawqrg    I’m not sure Bath is considered large It is in...
fhd9h84    Bath has quite a lot of Chinese students and a...
fhawwfz    Try screwfix and shops like that paint supplie...
Name: processed_text, dtype: object

In [ ]:
comments = c_df.sample(50)

comments['happy'], comments['angry'], comments['suprise'], comments['sad'], comments['fear'] = get_emotions([*comments['processed_text']])

comments.sample(5)

In [ ]:
### REFORMAT DATAFRAMES

c_df = c_df[['datetime', 'body', 'processed_text', 'sentiment', 'compound_score', 'submission_id', 'parent_id']]
s_df = s_df[['datetime', 'title', 'selftext']]

In [ ]:
### SAVE PROCESSED DATA

c_df.to_pickle(path + 'OneDrive - Cardiff University/Individual Project/processed data/comments.csv')
s_df.to_pickle(path + 'OneDrive - Cardiff University/Individual Project/processed data/submissions.csv')